In [2]:
import pandas as pd

file_path319 = 'A319.pkl'
file_path320 = 'A320.pkl'
file_path321 = 'A321.pkl'
file_path332 = 'A332.pkl'


df319 = pd.read_pickle(file_path319)
df320 = pd.read_pickle(file_path320)
df321 = pd.read_pickle(file_path321)
df332 = pd.read_pickle(file_path332)



In [28]:
gb = df319.groupby('fromICAO')
print(gb['segment'])

In [6]:
# List of dataframes to process
dfs = [df319, df320, df321, df332]
results = {}
sorteddicts = []

# Loop over each dataframe
for idx, df in enumerate(dfs, start=319):
    # Group by 'fromICAO' and create the dictionary
    gb = df.groupby('fromICAO')
    icao_dict = gb['segment'].unique().to_dict()

    # Initialize the lists to store results
    airports = []
    flights = []

    # Process the data
    total = 0
    for keys, values in icao_dict.items():
        amount = len(values)
        total += amount
        airports.append(keys)
        flights.append(amount)

    # Create a dictionary for flights and sort by the number of flights
    flights_dict = dict(zip(airports, flights))
    sorted_flights = dict(sorted(flights_dict.items(), key=lambda item: item[1], reverse=True))
    sorteddicts.append(sorted_flights)
    # Store only the top 5 results
    results[f'df{idx}'] = dict(list(sorted_flights.items())[:5])

# Print the top 5 sorted results for each dataframe
for df_name, top_5_flights in results.items():
    print(f"{df_name} top 5 sorted results:")
    print(top_5_flights)




df319 top 5 sorted results:
{28: 5194, 42: 3809, 16: 3574, 21: 3372, 35: 3029}
df320 top 5 sorted results:
{59: 8501, 159: 8139, 85: 7813, 37: 6845, 54: 6654}
df321 top 5 sorted results:
{9: 5856, 31: 5551, 14: 5254, 4: 4662, 32: 3513}
df322 top 5 sorted results:
{115: 1488, 10: 1271, 1: 1043, 23: 798, 28: 727}


In [36]:
!pip install folium

In [7]:
import folium

# List of aircraft labels and their corresponding colors
aircraft_labels = ['A319', 'A320', 'A321', 'A332']
marker_colors = ['blue', 'green', 'orange', 'red']  # Colors for each aircraft type

# Create a base map
base_map = folium.Map(location=[0, 0], zoom_start=2)  # Center the map globally

for i in range(len(sorteddicts)):
    lat_lon_dict = {}

    # Get the top 5 keys (segments) from the sorted dictionary
    top_5_fromICAO = list(sorteddicts[i].keys())[:5]

    for fromICAO in top_5_fromICAO:
        # Filter the dataframe where 'fromICAO' matches the current key
        filtered_row = df319[df319['fromICAO'] == fromICAO]
    
        if not filtered_row.empty:
            # Get the first 'lat' and 'lon' values from the filtered rows
            first_lat = float(filtered_row['lat'].iloc[0])  # Convert to float
            first_lon = float(filtered_row['lon'].iloc[0])  # Convert to float

            # Store the (lat, lon) tuple in the dictionary for this key
            lat_lon_dict[fromICAO] = (first_lat, first_lon)

            # Get the number of departing flights for the current fromICAO
            num_flights = sorteddicts[i][fromICAO]

            # Determine the color for the marker based on the aircraft type
            marker_color = marker_colors[i]

            # Add a marker for the coordinates on the map
            folium.Marker(
                location=[first_lat, first_lon],
                popup=f"{aircraft_labels[i]}: {fromICAO} ({first_lat}, {first_lon})\nDepartures: {num_flights}",
                icon=folium.Icon(color=marker_color)  # Use the determined color for the marker
            ).add_to(base_map)

    # Optionally, save the map for each aircraft type
    base_map.save(f"map_{aircraft_labels[i]}.html")

# Display the base map in a Jupyter notebook (if using Jupyter)
base_map
